# 3-qubit teleportation post-selection with gate + T1/Tphi noise

This notebook mirrors the post-selected teleportation experiment (no feed-forward, keep only measurement outcome `s=0`) and augments it with the same gate/readout plus T1/Tphi damping noise model used in the feed-forward study.


In [1]:
import math

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import DensityMatrix, partial_trace, state_fidelity
from qiskit_aer import AerSimulator
from qiskit_aer.noise import (
    NoiseModel,
    ReadoutError,
    amplitude_damping_error,
    depolarizing_error,
    phase_damping_error,
)


In [2]:
def combined_damping_error(t1=None, tphi=None, gate_time=2e-7):
    """Single-qubit channel capturing amplitude damping (T1) and pure dephasing (Tphi)."""
    errors = []
    if t1:
        p_amp = 1 - math.exp(-gate_time / t1)
        errors.append(amplitude_damping_error(p_amp))
    if tphi:
        p_phase = 1 - math.exp(-gate_time / tphi)
        errors.append(phase_damping_error(p_phase))

    if not errors:
        return None

    channel = errors[0]
    for err in errors[1:]:
        channel = channel.compose(err)
    return channel


In [3]:
def build_gate_noise_model(
    p_single=1e-3,
    p_two=5e-3,
    p_readout=1.5e-2,
    t1=150e-6,
    tphi=200e-6,
    gate_time=2e-7,
):
    noise_model = NoiseModel()

    single_dep = depolarizing_error(p_single, 1)
    two_dep = depolarizing_error(p_two, 2)
    readout_error = ReadoutError([[1 - p_readout, p_readout], [p_readout, 1 - p_readout]])

    damping = combined_damping_error(t1=t1, tphi=tphi, gate_time=gate_time)
    if damping:
        single_quantum_error = single_dep.compose(damping)
        two_quantum_error = two_dep.compose(damping.tensor(damping))
    else:
        single_quantum_error = single_dep
        two_quantum_error = two_dep

    single_gate_set = ["id", "x", "sx", "rz", "h"]
    two_gate_set = ["cx", "cz"]

    noise_model.add_all_qubit_quantum_error(single_quantum_error, single_gate_set)
    noise_model.add_all_qubit_quantum_error(two_quantum_error, two_gate_set)
    noise_model.add_all_qubit_readout_error(readout_error)
    return noise_model


In [4]:
def original_pair_circuit():
    circ = QuantumCircuit(3)
    circ.h(0)
    circ.cx(0, 1)
    circ.save_density_matrix(label="rho_orig")
    return circ


def postselection_circuit():
    tele_circ = QuantumCircuit(3, 1)
    tele_circ.h(0)
    tele_circ.cx(0, 1)
    tele_circ.h(2)
    tele_circ.cz(1, 2)
    tele_circ.h(1)
    tele_circ.measure(1, 0)
    tele_circ.h(2)
    tele_circ.save_density_matrix(label="rho_final", conditional=True)
    return tele_circ


def simulate_density_matrix(circuit, simulator, label, conditional=False):
    compiled = transpile(circuit, simulator)
    result = simulator.run(compiled).result()
    data = result.data(0)[label]
    if conditional:
        return {k: DensityMatrix(v) for k, v in data.items()}
    return DensityMatrix(data)


def postselected_density_matrix(simulator, key="0x0"):
    rho_dict = simulate_density_matrix(postselection_circuit(), simulator, "rho_final", conditional=True)
    dm = rho_dict.get(key)
    if dm is None:
        raise RuntimeError(f"Measurement outcome {key} not present in simulation data: {list(rho_dict)}")
    prob = dm.trace().real
    return dm / prob, prob


In [5]:
noise_params = {
    "p_single": 2e-3,
    "p_two": 1e-2,
    "p_readout": 1.5e-2,
    "t1": 120e-6,
    "tphi": 180e-6,
    "gate_time": 250e-9,
}

ideal_sim = AerSimulator(method="density_matrix")
noise_model = build_gate_noise_model(**noise_params)
noisy_sim = AerSimulator(method="density_matrix", noise_model=noise_model)

rho_full_orig = simulate_density_matrix(original_pair_circuit(), ideal_sim, "rho_orig")
rho_01 = partial_trace(rho_full_orig, [2])

rho_post_ideal, prob_ideal = postselected_density_matrix(ideal_sim, key="0x0")
rho_02_ideal = partial_trace(rho_post_ideal, [1])

rho_post_noisy, prob_noisy = postselected_density_matrix(noisy_sim, key="0x0")
rho_02_noisy = partial_trace(rho_post_noisy, [1])

print("Noise parameters:")
for k, v in noise_params.items():
    print(f"  {k}: {v}")
print(f"\nSuccess probability (ideal) for outcome s=0: {prob_ideal:.4f}")
print(f"Success probability (noisy) for outcome s=0: {prob_noisy:.4f}")
print("\nFidelity (ideal post-selected pair vs original):", state_fidelity(rho_01, rho_02_ideal))
print("Fidelity (noisy post-selected pair vs original):", state_fidelity(rho_01, rho_02_noisy))


Noise parameters:
  p_single: 0.002
  p_two: 0.01
  p_readout: 0.015
  t1: 0.00012
  tphi: 0.00018
  gate_time: 2.5e-07

Success probability (ideal) for outcome s=0: 1.0000
Success probability (noisy) for outcome s=0: 1.0000

Fidelity (ideal post-selected pair vs original): 1.0000000000000004
Fidelity (noisy post-selected pair vs original): 0.9451466601717223


In [6]:
print("\nT1 / Tphi sweep for post-selected fidelity:")
for t1_us in [80, 120, 200]:
    params = dict(noise_params)
    params["t1"] = t1_us * 1e-6
    params["tphi"] = (t1_us + 40) * 1e-6
    nm = build_gate_noise_model(**params)
    sim = AerSimulator(method="density_matrix", noise_model=nm)
    rho_post, prob = postselected_density_matrix(sim, key="0x0")
    rho_02 = partial_trace(rho_post, [1])
    fidelity = state_fidelity(rho_01, rho_02)
    print(f"  T1={t1_us:>3} us, Tphi={t1_us + 40:>3} us, success={prob:.4f} -> fidelity={fidelity:.6f}")



T1 / Tphi sweep for post-selected fidelity:
  T1= 80 us, Tphi=120 us, success=1.0000 -> fidelity=0.956483
  T1=120 us, Tphi=160 us, success=1.0000 -> fidelity=0.958687
  T1=200 us, Tphi=240 us, success=1.0000 -> fidelity=0.965053
